# 飞桨常规赛：PALM眼底彩照中黄斑中央凹定位 - 12月第3名方案


# （1）比赛介绍

## 赛题介绍



PALM黄斑定位常规赛的重点是研究和发展与患者眼底照片黄斑结构定位相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上定位黄斑的自动算法。具体目的是预测黄斑中央凹在图像中的坐标值。

![](https://ai-studio-static-online.cdn.bcebos.com/caac4481a304405db9e5c4ce14497c029ed4ca5d06b6485cb4decd97cbbd136a)


中央凹是视网膜中辨色力、分辨力最敏锐的区域。以人为例，在视盘颞侧约3.5mm处，有一黄色小区，称黄斑，其中央的凹陷，就是中央凹。中央凹的准确定位可辅助医生完成糖尿病视网膜、黄斑变性等病变的诊断。







# （2）数据介绍
PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带黄斑中央凹坐标标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。

## 数据说明
本次常规赛提供的金标准由中山大学中山眼科中心的7名眼科医生手工进行标注，之后由另一位高级专家将它们融合为最终的标注结果。本比赛提供数据集对应的黄斑中央凹坐标信息存储在xlsx文件中，名为“Fovea_Location_train”，第一列对应眼底图像的文件名(包括扩展名“.jpg”)，第二列包含x坐标，第三列包含y坐标。
图

## 训练数据集
文件名称：Train
Train文件夹里有一个文件夹fundus_images和一个xlsx文件。

fundus_images文件夹内包含800张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、P0001.jpg、N0001.jpg和V0001.jpg。
xlsx文件中包含800张眼底彩照对应的x、y坐标信息。
## 测试数据集
文件名称：PALM-Testing400-Images 文件夹里包含400张眼底彩照，命名形如T0001.jpg。

# 一、数据处理
## 1.1 解压数据集

In [ ]:
!unzip -oq /home/aistudio/data/data116960/常规赛：PALM眼底彩照中黄斑中央凹定位.zip
!mv │г╣ц╚№г║PALM╤█╡╫▓╩╒╒╓╨╗╞░▀╓╨╤ы░╝╢и╬╗ 常规赛：PALM眼底彩照中黄斑中央凹定位
!rm -rf __MACOSX


In [ ]:
!mv /home/aistudio/PALM眼底彩照中黄斑中央凹定位/* /home/aistudio/work/常规赛：PALM眼底彩照中黄斑中央凹定位/


## 1.2 查看数据标签

In [ ]:
import blackhole.dataframe as pd
df=pd.read_excel('常规赛：PALM眼底彩照中黄斑中央凹定位/Train/Fovea_Location_train.xlsx')

df.head()

,imgName,Fovea_X,Fovea_Y
0,H0001.jpg,743.96,790.54
1,H0002.jpg,1394.82,725.54
2,H0003.jpg,1361.74,870.72
3,H0004.jpg,703.15,742.44
4,H0005.jpg,1070.95,1037.54


In [ ]:
# 计算标签的均值和标准差，用于标签的归一化
key_pts_values = df.values[:,1:] # 取出标签信息
data_mean = key_pts_values.mean() # 计算均值
data_std = key_pts_values.std() # 计算标准差


print('标签的均值为:', data_mean)
print('标签的标准差为:', data_std)

标签的均值为: 1085.6073687500023
标签的标准差为: 183.5345073716085


## 1.3 数据增强

* 为了防止过拟合和泛化能力不强，对图片进行数据增强处理

* 本案例使用的主要操作包括剪裁和灰度化等等


In [ ]:
import paddle.vision.transforms.functional as F
class Resize(object):
    # 将输入图像调整为指定大小

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, data):

        image = data[0]    # 获取图片
        key_pts = data[1]  # 获取标签

        image_copy = np.copy(image)      
        key_pts_copy = np.copy(key_pts)

        h, w = image_copy.shape[:2]

        new_h, new_w = self.output_size,self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = F.resize(image_copy, (new_h, new_w))
        
        # scale the pts, too
        #key_pts_copy[::2] = key_pts_copy[::2] * new_w / w
        #key_pts_copy[1::2] = key_pts_copy[1::2] * new_h / h

        return img, key_pts_copy




class GrayNormalize(object):
    # 将图片变为灰度图，并将其值放缩到[0, 1]
    # 将 label 放缩到 [-1, 1] 之间

    def __call__(self, data):
        image = data[0]   # 获取图片
        key_pts = data[1] # 获取标签
        
        image_copy = np.copy(image)
        key_pts_copy = np.copy(key_pts)

        # 灰度化图片
        gray_scale = paddle.vision.transforms.Grayscale(num_output_channels=3)
        image_copy = gray_scale(image_copy)
        
        # 将图片值放缩到 [0, 1]
        image_copy = (image_copy-127.5) / 127.5
        
        # 将坐标点放缩到 [-1, 1]
        #mean = data_mean # 获取标签均值
        #std = data_std   # 获取标签标准差

        #key_pts_copy = (key_pts_copy - mean)/std

        return image_copy, key_pts_copy

class ToCHW(object):
    # 将图像的格式由HWC改为CHW
    def __call__(self, data):

        image = data[0]
        key_pts = data[1]

        transpose = T.Transpose((2, 0, 1)) # 改为CHW
        image = transpose(image)
        
        return image, key_pts

In [ ]:
import paddle.vision.transforms as T
data_transform = T.Compose([
                        Resize(224),
                        
                        GrayNormalize(),
                        ToCHW(),
                         ])
data_transform2 = T.Compose([
                        Resize(224),
                        GrayNormalize(),
                        ToCHW(),
                        ])

## 1.4 自定义数据集

In [ ]:
path='常规赛：PALM眼底彩照中黄斑中央凹定位/Train/fundus_image/'
df=df.sample(frac=1)
image_list=[]
label_listx=[]
label_listy=[]

for i in range(len(df)):
        image_list.append(path+df['imgName'][i])
        label_listx.append(df['Fovea_X'][i])
        label_listy.append(df['Fovea_Y'][i])

In [ ]:

import os
test_path='常规赛：PALM眼底彩照中黄斑中央凹定位/PALM-Testing400-Images'
test_list=[]
test_labelx=[]
test_labely=[]

list = os.listdir(test_path)  # 列出文件夹下所有的目录与文件
for i in range(0, len(list)):
    path = os.path.join(test_path, list[i])
    test_list.append(path)
    test_labelx.append(0)
    test_labely.append(0)


In [ ]:
import paddle
import cv2
import numpy as np
class dataset(paddle.io.Dataset):
    def __init__(self,img_list,label_listx,label_listy,transform=None,transform2=None,mode='train'):

        self.image=img_list
        self.labelx=label_listx
        self.labely=label_listy
        self.mode=mode
        self.transform=transform
        self.transform2=transform2
    def load_img(self, image_path):

        img=cv2.imread(image_path,1)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        h,w,c=img.shape
        return img,h,w


    def __getitem__(self,index):
        img,h,w = self.load_img(self.image[index])
        labelx = self.labelx[index]/w
        labely = self.labely[index]/h
        img_size=img.shape

        if self.transform:
            if self.mode=='train':
                img, label = self.transform([img, [labelx,labely]])
            else:
                img, label = self.transform2([img, [labelx,labely]])
        
        
        label=np.array(label,dtype='float32')
        img=np.array(img,dtype='float32')
        return img,label

    def __len__(self):
        return len(self.image)

## 1.5 划分数据集

* 以0.85的比率划分训练集和测试集，数据集中有85%作为训练集，15%作为验证集

In [ ]:
radio=0.85
train_list=image_list[:int(len(image_list)*radio)]
train_labelx=label_listx[:int(len(label_listx)*radio)]
train_labely=label_listy[:int(len(label_listy)*radio)]


val_list=image_list[int(len(image_list)*radio):]
val_labelx=label_listx[int(len(label_listx)*radio):]
val_labely=label_listy[int(len(label_listy)*radio):]


train_ds=dataset(train_list,train_labelx,train_labely,data_transform,data_transform2,'train')
val_ds=dataset(val_list,val_labelx,val_labely,data_transform,data_transform2,'valid')
test_ds=dataset(test_list,test_labelx,test_labely,data_transform,data_transform2,'test')

## 1.6 查看图片

In [ ]:
import matplotlib.pyplot as plt
for i,data in enumerate(train_ds):
    
    img,label=data

    img=img.transpose([1,2,0])
    print(img.shape)
    
    plt.title(label)
    plt.imshow(img)
    plt.show()

    if i==0:
        break

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'col

(224, 224, 3)


<Figure size 640x480 with 1 Axes>

# 二、模型构建
## 2.1 模型组网
* 本案例选择resnet152进行模型组网，resnet152相比于resnet50对于本赛题的准确性更高
* 详情可参考[官方文档](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/02_paddle2.0_develop/04_model_cn.html)组建神经网络模型

In [ ]:
class MyNet(paddle.nn.Layer):
    def __init__(self):
        super(MyNet, self).__init__()
        self.resnet = paddle.vision.resnet152(pretrained=True, num_classes=0) 
        self.flatten = paddle.nn.Flatten()
        self.linear_1 = paddle.nn.Linear(2048, 512)
        self.linear_2 = paddle.nn.Linear(512, 256)
        self.linear_3 = paddle.nn.Linear(256, 2)
        self.relu = paddle.nn.ReLU()
        self.dropout = paddle.nn.Dropout(0.2)
    
    def forward(self, inputs):

        y = self.resnet(inputs)
        y = self.flatten(y)
        y = self.linear_1(y)
        y = self.linear_2(y)
        y = self.relu(y)
        y = self.dropout(y)
        y = self.linear_3(y)
        y = paddle.nn.functional.sigmoid(y)

        return y

## 2.2 异步加载数据

In [ ]:
train_loader = paddle.io.DataLoader(train_ds, places=paddle.CPUPlace(), batch_size=32, shuffle=True, num_workers=0)
val_loader = paddle.io.DataLoader(val_ds, places=paddle.CPUPlace(), batch_size=32, shuffle=True, num_workers=0)
test_loader=paddle.io.DataLoader(test_ds, places=paddle.CPUPlace(), batch_size=32, shuffle=False, num_workers=0)

## 2.3 自定义损失函数

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances 
import paddle.nn as nn
# 损失函数
def cal_coordinate_Loss(logit, label, alpha = 0.5):
    """
    logit: shape [batch, ndim]
    label: shape [batch, ndim]
    ndim = 2 represents coordinate_x and coordinaate_y
    alpha: weight for MSELoss and 1-alpha for ED loss
    return: combine MSELoss and ED Loss for x and y, shape [batch, 1]
    """
    alpha = alpha
    mse_loss = nn.MSELoss(reduction='mean')

    mse_x = mse_loss(logit[:,0],label[:,0])
    mse_y = mse_loss(logit[:,1],label[:,1])
    mse_l = 0.5*(mse_x + mse_y)
    # print('mse_l', mse_l)

    ed_loss = []
    # print(logit.shape[0])
    for i in range(logit.shape[0]):
        logit_tmp = logit[i,:].numpy()
        label_tmp = label[i,:].numpy()
        # print('cal_coordinate_loss_ed', logit_tmp, label_tmp)        
        ed_tmp = euclidean_distances([logit_tmp], [label_tmp])
        # print('ed_tmp:', ed_tmp[0][0])
        ed_loss.append(ed_tmp)
    
    ed_l = sum(ed_loss)/len(ed_loss)
    # print('ed_l', ed_l)
    # print('alpha', alpha)
    loss = alpha * mse_l + (1-alpha) * ed_l
    # print('loss in function', loss)
    return loss

In [ ]:
class SelfDefineLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(SelfDefineLoss, self).__init__()

   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用PaddlePaddle中相关API自定义的计算逻辑
       output = cal_coordinate_Loss(input,label)
       return output

# 三、训练、评估与预测
## 3.1 模型训练与可视化

* 本次比赛结果的调优过程：设定了401轮迭代（从epoch1到epoch401）

* 批次大小Batch_size设定为32，根据自己的显存大小进行设定

* 因为计算机字符都是以2的指数次幂进行存储的，所以设置 batch_size时尽量选择例如 4， 8， 16， 32， 64， 128， 256 等

* Batch Size的大小影响模型的优化程度和速度。同时其直接影响到GPU内存的使用情况，假如你GPU内存不大，该数值最好设置小一点

* 使用 cosine annealing 的策略来动态调整学习率，learning_rate为1e-5

* 经测试以上参数设定可以达到较好的结果

In [16]:
from utils import NME
visualdl=paddle.callbacks.VisualDL(log_dir='visual_log')
#定义输入

Batch_size=32
EPOCHS=401
step_each_epoch = len(train_ds)//Batch_size


# 使用 paddle.Model 封装模型
model = paddle.Model(MyNet())

#模型加载
#model.load('/home/aistudio/work/lup/final')


lr = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=1e-5,
                                                T_max=step_each_epoch * EPOCHS)




# 定义Adam优化器
optimizer = paddle.optimizer.Adam(learning_rate=lr,
                                weight_decay=1e-5,
                                parameters=model.parameters())
# 定义SmoothL1Loss
loss =paddle.nn.SmoothL1Loss()
#loss =SelfDefineLoss()

# 使用自定义metrics
metric = NME()

model.prepare(optimizer=optimizer, loss=loss, metrics=metric)




# 训练可视化VisualDL工具的回调函数

# 启动模型全流程训练
model.fit(train_loader,  # 训练数据集
          val_loader,   # 评估数据集
          epochs=EPOCHS,       # 训练的总轮次
          batch_size=Batch_size,  # 训练使用的批大小
          save_dir="/home/aistudio/work/lup", #把模型参数、优化器参数保存至自定义的文件夹
          save_freq=20,                    #设定每隔多少个epoch保存模型参数及优化器参数
          verbose=1 ,      # 日志展示形式
          callbacks=[visualdl]
          )  # 设置可视化

100%|██████████| 355826/355826 [00:10<00:00, 35415.15it/s]


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/401


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


## 3.2 模型评估

* 最终比赛提交的结果中，checkpoints使用的是/home/aistudio/work/lup/路径下的**final**

* 通过以下操作进行模型评估

In [17]:
# 模型评估
model.load('/home/aistudio/work/lup/final')
result = model.evaluate(val_loader, verbose=1)
print(result)

Eval begin...
step 4/4 [==============================] - loss: 0.0015 - nme: 0.0507 - 2s/step
Eval samples: 120
{'loss': [0.0014828634], 'nme': 0.05066016193988991}


## 3.3 预测结果

In [18]:
# 进行预测操作
result = model.predict(test_loader)

Predict begin...
step 13/13 [==============================] - 2s/step         
Predict samples: 400


In [19]:
# 获取测试图片尺寸和图片名
test_path='常规赛：PALM眼底彩照中黄斑中央凹定位/PALM-Testing400-Images'
test_size=[]
FileName=[]
for i in range(len(list)):
    path = os.path.join(test_path, list[i])
    img=cv2.imread(path,1)
    test_size.append(img.shape)
    FileName.append(list[i])
test_size=np.array(test_size)

* 将预测结果写入result.csv文件中

In [20]:
result=np.array(result)
pred=[]
for i in range(len(result[0])):
    pred.extend(result[0][i])
pred=np.array(pred) 

pred = paddle.to_tensor(pred)
out=np.array(pred).reshape(-1,2)

#Fovea_X=out[:,0]*data_std+data_mean
#Fovea_Y=out[:,1]*data_std+data_mean
Fovea_X=out[:,0]
Fovea_Y=out[:,1]

Fovea_X=Fovea_X*test_size[:,1]
Fovea_Y=Fovea_Y*test_size[:,0]

submission = pd.DataFrame(data={
                            "FileName": FileName,
                            "Fovea_X": Fovea_X,
                            "Fovea_Y": Fovea_Y
                        })
submission=submission.sort_values(by='FileName')
submission.to_csv("result.csv", index=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


# 总结

* 进一步调整学习率及超参数

* 尝试其他优化器

* 尝试定义更深层的神经网络

* 模型能力有限度，尝试新的网络模型

* 丰富数据增强操作


# 参考资料

[飞桨常规赛：PALM眼底彩照中黄斑中央凹定位-9月第1名方案](https://aistudio.baidu.com/aistudio/projectdetail/2332679)

[飞桨常规赛：PALM眼底彩照中黄斑中央凹定位-8月第9名方案](https://aistudio.baidu.com/aistudio/projectdetail/2309957?channelType=0&channel=0)

[『深度学习7日打卡营』人脸关键点检测](https://aistudio.baidu.com/aistudio/projectdetail/1487972)
